# Practica para Sistemas de Bases de Datos (Feb 21)

## Autor: Martín Romera Sobrado
## Contacto: mromera95@alumno.uned.es

### Inicialización de la base de datos

Creamos e iniciamos la base de datos si todavía no lo está.

In [24]:
! hadoop fs -mkdir librosdb

mkdir: `librosdb': File exists


In [25]:
! beeline -u "jdbc:hive2://localhost:10000/default" -f db/db.hql

scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.12.0)
Driver: Hive JDBC (version 1.1.0-cdh5.12.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
que no este creada.alhost:10000/default> -- Crea la base de datos en el caso de  
0: jdbc:hive2://localhost:10000/default> 
0: jdbc:hive2://localhost:10000/default> create database if not exists librosdb
de libros'. . . . . . . . . . . . . . .> Comment 'Base de datos con información  
. . . . . . . . . . . . . . . . . . . .> Location '/home/cloudera/librosdb'
n Romera Sobrado', 'Creada el' = '9-Nov-2020');bproperties('Creada por' = 'Martí 
INFO  : Compiling command(queryId=hive_20201109085151_eaa70c4f-ded6-4aad-8b26-f46a56a47f5e): create database if not exists librosdb
Comment 'Base de datos con información de libros'
Location '/home/cloudera/librosdb'
With dbproperties('Creada por' = 'Martín Romera Sobrado', 'Creada el' = '9-Nov-2020')
INFO  : Semantic Analysis Complete

Los contenidos de *db/db.hql* son los siguientes:

```hive
-- Crea la base de datos en el caso de que no este creada.

create database if not exists librosdb
Comment 'Base de datos con información de libros'
Location '/home/cloudera/librosdb'
With dbproperties('Creada por' = 'Martín Romera Sobrado', 'Creada el' = '9-Nov-2020');

```

### Ejercicios

Ej 1. *Crea las tablas necesarias para almacenar los datos. Pueden ser internas o externas en función de los datos que se desee. La decisión de interna o externa debe estar razonada.*

Las tablas serán externas, ya que en el ejercicio 5 tendremos un programa *MapReduce* en python que acceda a los datos.

In [26]:
! beeline -u "jdbc:hive2://localhost:10000/default" -f db/ej1.hql

scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.12.0)
Driver: Hive JDBC (version 1.1.0-cdh5.12.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
tasetsc:hive2://localhost:10000/default> -- Creamos las dos tablas: authors y da 
0: jdbc:hive2://localhost:10000/default> 
0: jdbc:hive2://localhost:10000/default> USE librosdb;
INFO  : Compiling command(queryId=hive_20201109085151_7499d642-a0d0-4249-a985-bb05509d18d8): USE librosdb
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=hive_20201109085151_7499d642-a0d0-4249-a985-bb05509d18d8); Time taken: 0.044 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20201109085151_7499d642-a0d0-4249-a985-bb05509d18d8): USE librosdb
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing comman

Los contenidos de *db/ej1.hql* son los siquientes:
```hive
-- Creamos las dos tablas: authors y datasets

USE librosdb;

-- Tabla authors

CREATE EXTERNAL TABLE IF NOT EXISTS authors
(
    author_id INT,
    author_name STRING
)
COMMENT 'Tabla de autores';

-- Tabla datasets

CREATE EXTERNAL TABLE IF NOT EXISTS datasets
(
    title STRING,
    author_id INT,
    bestsellers_rank INT,
    imprint BINARY,
    publication_date TIMESTAMP,
    rating_avg DOUBLE,
    rating_count INT
)
COMMENT 'Tabla de libros'
```

Ej 2. *Importa los datos en las tablas creadas*

In [27]:
! hadoop fs -put db/csv/authors.csv librosdb

put: `librosdb/authors.csv': File exists


In [28]:
! hadoop fs -put db/csv/dataset.csv librosdb

put: `librosdb/dataset.csv': File exists
